In [ ]:
import numpy as np
import pickle
from pathlib import Path
from openai import AzureOpenAI
from faiss_index import *

In [ ]:
def query(q, index, path):
    client = AzureOpenAI(
        azure_deployment = "ada_embedding"
        api_version = "2023-05-15"
        azure_endpoint = os.environ["OPENAI_MODEL_ENDPOINT"]
        api_key = os.environ["OPENAI_API_KEY"]
    )
    response = client.embeddings.create(
            model = MODEL_DEPLOYMENT_NAME,
            input = q
    )
    q_embedding = np.array([response.data[0].embedding])
    results = index.search(q_embedding, 3)
    docs = results[1][0]
    scores = results[0][0]
    with open(str(Path(path) / "index.pkl"), 'rb') as f:
        docstore, index_to_docstore_id = pickle.load(f)
    for i in range(len(docs)):
        d = docstore.search(index_to_docstore_id.get(docs[i]))
        print(f"Source: {d.metadata.get('source')}")
        print(f"{d.metadata.get('title')}")
        print(f"Content: {d.page_content}")
        print(f"Score: {scores[i]}")
        print("=" * 50)

In [ ]:
index = create_indexIVF(f"C:/Users/DAXIAO/OneDrive - HC-SC PHAC-ASPC/Documents/10k_sample/subset_1", "faiss_index_IVF", True)

In [ ]:
index = faiss.read_index("faiss_index_IVF/index.faiss")

In [ ]:
q = "Vietnamese language generation with LLMs"
query(q, index, "faiss_index_IVF")